In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# importing required libraries

In [ ]:
import os 
import sys
import torch 
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from PIL import Image
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")
from timm import create_model
from fastai.vision.all import *

# Exploring dataset 

In [ ]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sub_df = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

In [ ]:
# checking if the values are all in jpg format 
def check_values(mode="train"):
    value= 0 
    total_vals = os.listdir(f"../input/petfinder-pawpularity-score/{mode}")
    for x in os.listdir(f"../input/petfinder-pawpularity-score/{mode}"):
        x = x[-3:]

        if x == "jpg":
            value += 1 
        else : 
            print(f'another type ${x}')
    print(f"total values ${len(total_vals)} \n and jpg values ${value}")

check_values()
check_values(mode="test")
        

In [ ]:
def get_whole_path(id, mode = "train"):
    
    before_url = f"../input/petfinder-pawpularity-score/{mode}/"
    return before_url + id + ".jpg"

In [ ]:
Image.open(get_whole_path(train_df['Id'][0]))

In [ ]:
train_df['Path'] = train_df['Id'].apply(lambda x: get_whole_path(id = x , mode = "train"))
train_df = train_df.drop("Id", axis=1)
train_df = train_df.sample(frac = 1).reset_index(drop = True)
train_df.head()


In [ ]:
# checking the distribution for pawpularity scores 
train_df['Pawpularity'].plot(kind= "hist")
plt.xlabel("pawpularity score")
print(f"mean of the pawpularity is {train_df['Pawpularity'].mean()}")
print(f"median of the pawpularity is {train_df['Pawpularity'].median()}")
print(f"standard deviation of pawpularity is {train_df['Pawpularity'].std()}")

In [ ]:
# torch.log2(torch.tensor(len(train_df)))
no_bins = int(np.ceil(2*(len(train_df)**(1/3)) ))
no_bins


train_df["Norm"] = train_df['Pawpularity']/100

In [ ]:
train_df['bins'] = pd.cut(train_df['Pawpularity'], bins=no_bins, labels=False)
train_df['bins'].hist()

In [ ]:
train_df.head()

In [ ]:
# creating stratified k folds 
train_df['Fold'] = -1 
n_folds = 10 
stratified_k_folds = StratifiedKFold(n_splits=n_folds, shuffle = True )
for i, (shi, train_index) in enumerate(stratified_k_folds.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
train_df['Fold'] = train_df['Fold'].astype('int')
train_df.Fold.value_counts().plot.bar()

In [ ]:
train_df[train_df['Fold']==1].head()

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
def get_data(fold):
#     train_df_no_val = train_df.query(f'fold != {fold}')
#     train_df_val = train_df.query(f'fold == {fold}')
    
#     train_df_bal = pd.concat([train_df_no_val,train_df_val.sample(frac=1).reset_index(drop=True)])
    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['Fold'] == fold)
    
    dls = ImageDataLoaders.from_df(train_df_f, #pass in train DataFrame
#                                valid_pct=0.2, #80-20 train-validation random split
                               valid_col='is_valid', #
                               seed=365, #seed
                               fn_col='Path', #filename/path is in the second column of the DataFrame
                               label_col='Norm', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=32, #pass in batch size
                               num_workers=4,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms
    
    return dls

In [ ]:
#Valid Kfolder size
the_data = get_data(0)
assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//32)

In [ ]:
def get_learner(fold_num):
    data = get_data(fold_num)
    
    model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)

    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    
    return learn

In [ ]:
test_df.head()

In [ ]:
test_df['Pawpularity'] = [1]*len(test_df)
test_df['Path'] = test_df['Id'].map(lambda x:get_whole_path(id = x , mode = "test"))
test_df = test_df.drop(columns=['Id'])
test_df['Norm'] = test_df['Pawpularity']/100


In [ ]:
test_df

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
get_learner(fold_num=0).lr_find(end_lr=3e-2)

In [ ]:
import gc 
all_preds = []

for i in range(n_folds):

    print(f'Fold {i} results')
    
    learn = get_learner(fold_num=i)

    learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=3)]) 
    
    learn.recorder.plot_loss()

    #learn = learn.to_fp32()
    if i == n_folds - 1:
#         learn.export(f'model_fold_{i}.pkl')
        learn.save(f'model_fold_{i}.pkl')
    
    
    dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=365, #seed
                               fn_col='Path', #filename/path is in the second column of the DataFrame
                               label_col='Norm', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=32, #pass in batch size
                               num_workers=4,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
    
    test_dl = dls.test_dl(test_df)
    
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    
    all_preds.append(preds)
    
    del learn

    torch.cuda.empty_cache()

    gc.collect()

In [ ]:
all_preds

In [ ]:
np.mean(np.stack(all_preds*100))

In [ ]:
sample_df = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
preds = np.mean(np.stack(all_preds), axis=0)
sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv('submission.csv').head()